In [ ]:
import pandas as pd
import csv
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.utils import resample, shuffle
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import accuracy_score
from sklearn import datasets
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import label_binarize
from sklearn.preprocessing import StandardScaler
from sklearn import datasets
import os


import time
from scipy.stats import spearmanr
from sklearn.model_selection import GridSearchCV

import os
import sys
import subprocess
import seaborn as sns
from sklearn.inspection import permutation_importance
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score

from sklearn.model_selection import ShuffleSplit
import joblib
import glob

import json
import shutil
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
import pickle

from boruta import BorutaPy
from copy import deepcopy
import shap 

In [ ]:



"""
Combine results
"""



sp_rarefied_name_list = ['San','Rub','Rec','Pro','Mex','Maz','Lon','Lec','Ind','Ger','Dim']

buffer_name_list = ['b_05']

rarefied_name_list = [''] #using the 5km rarefied data. Leave this blank if you don't have special notation on your file name for the rarefaction.

for rarefied_name in rarefied_name_list:
    for buffer_name in buffer_name_list:
        for bug_name in sp_rarefied_name_list:

            grid_name = '5' # We are using 0.05˚ grid
            
            name_for_saving = bug_name+rarefied_name+'_g'+grid_name+'_'+buffer_name 
            """
            modified
            """
            print(name_for_saving)

            buffer_path = '/Users/huangliting/Desktop/kissing_bugs_stats/buffers/'+bug_name+rarefied_name+'_g0'+grid_name+'_'+buffer_name+'.csv'
            buffer_select = pd.read_csv(buffer_path)
            buffer_select.drop(columns=['left', 'top', 'right', 'bottom'], inplace=True)


            presence_path = '/Users/huangliting/Desktop/kissing_bugs_stats/presence_point/'+bug_name+rarefied_name+'_g'+grid_name+'.csv'
            presence_select = pd.read_csv(presence_path)
            presence_select.drop(columns=['left', 'top', 'right', 'bottom'], inplace=True)


            zonal_path = '/Users/huangliting/Desktop/kissing_bugs_stats/zonal_stats_csv/'+grid_name+'km.csv'
            zonal_select = pd.read_csv(zonal_path)
            zonal_select.drop(columns=['Unnamed: 0','left', 'top', 'right', 'bottom','forest_grassland'], inplace=True)

            raw_df = pd.merge(pd.merge(buffer_select, presence_select, on='id'),zonal_select, on='id')


            """
            select presence data
            """
            df0 = raw_df.copy()

            species_data = df0.loc[df0['count'] ==1].copy()
            species_data= species_data.loc[~species_data['NFFD'].isnull()].copy()



            species_data = species_data.loc[~species_data['water'].isnull()].copy()
            species_data = species_data.loc[~species_data['cropland'].isnull()].copy()
            species_data = species_data.loc[~species_data['permanent_snow_ice'].isnull()].copy()
            species_data = species_data.loc[~species_data['barren'].isnull()].copy()
            species_data = species_data.loc[~species_data['urban'].isnull()].copy()
            species_data = species_data.loc[~species_data['grassland'].isnull()].copy()
            species_data = species_data.loc[~species_data['forest'].isnull()].copy()

            species_data['presence']=1

            species_data.drop(columns=['id','count', buffer_name], inplace=True)

            number_of_presence = species_data.shape[0]


            """
            select north america
            """
            df_na = raw_df.copy()
            background_data= df_na.loc[~df_na['NFFD'].isnull()].copy()


            background_data = background_data.loc[~background_data['water'].isnull()].copy()
            background_data = background_data.loc[~background_data['cropland'].isnull()].copy()
            background_data = background_data.loc[~background_data['permanent_snow_ice'].isnull()].copy()
            background_data = background_data.loc[~background_data['barren'].isnull()].copy()
            background_data = background_data.loc[~background_data['urban'].isnull()].copy()
            background_data = background_data.loc[~background_data['grassland'].isnull()].copy()
            background_data = background_data.loc[~background_data['forest'].isnull()].copy()


            testing_group_raw = background_data.copy()
            id_list = testing_group_raw['id']
            testing_group_raw.drop(['id','count', buffer_name], axis=1,inplace=True)
            final_testing_group = testing_group_raw.copy()


            """
            select absence pool
            """
            buffer_selection_raw = background_data.copy()
            buffer_background = buffer_selection_raw.loc[buffer_selection_raw[buffer_name].isnull()].copy()
            buffer_background['presence']=0
            buffer_background.drop(['id','count', buffer_name], axis=1,inplace=True)



            """
            define scoring
            """

            def tss_scorer(clf, X, y):
                y_pred = clf.predict(X)
                cm = confusion_matrix(y, y_pred,labels=[0,1])
                tss = (cm[0, 0]/(cm[0, 0]+cm[1, 0]))+(cm[1, 1]/(cm[1, 1]+cm[0, 1]))-1
                return tss


            """
            modify here
            """

            # Disable the printing
            def blockPrint():
                sys.stdout = open(os.devnull, 'w')

            # Restore
            def enablePrint():
                sys.stdout = sys.__stdout__

            num_runs = 50 #total number of iteration

            count_num = 0 #the current number of iteration

            count_average = 0 #for storing average cross-validation score

            array_list = np.array([]) #record permutation importance, importance
            variance_list = np.array([]) #record permutation importance, variance

            giniarray_list = np.array([]) #record gini importance, importance
            ginivariance_list = np.array([]) #record gini importance, variance

            predict_list = np.array([]) #record prediction of north america

            alldata_list = np.array([]) #record all input data of north america

            fpr_list = np.array([]) #record roc of north america

            tpr_list = np.array([]) #record roc of north america

            roc_auc_count = 0 #record auc of north america

            X_list = np.array([])
            y_list = np.array([]) #store the data used in each run

            boruta_rank = np.array([]) #store boruta rank in each run

            shape_values_list0 = np.array([])
            shape_values_list1 = np.array([])#store shapevalues
            x_train_list = np.array([])

            """change_name"""
            buffer_used = buffer_background.copy() #the buffer to select psudo-absence points

            size_group = species_data.shape[0]#number of presence data points


            x_labels = np.delete(species_data.keys(), np.argwhere(species_data.keys() == 'presence'))
            variable_length = len(x_labels) # for making sure the variables are arranged in the same order later 

            #create a directory for storing the trees
            save_tree_dir = '/Users/huangliting/Desktop/kissing_bugs_stats/trees/'+name_for_saving 


            if not os.path.exists(save_tree_dir):
                os.makedirs(save_tree_dir)


            x_train_length = 0

            while count_num < num_runs:

                buffer_thisround = buffer_used.copy()

                background_data = buffer_thisround.sample(n=size_group)
                species_thisround = species_data.copy()
                combine_list = [species_thisround,background_data]

                combined_df = pd.concat(combine_list)



                # Get the y values
                y=combined_df['presence']
                y=y.astype('int')

                # Get the x values
                X = combined_df.copy()
                X.drop(['presence'], axis=1,inplace=True)
                X.astype('float')

                X_list = np.append(X_list, X)
                y_list = np.append(y_list, y)




                # Split dataset into training set and test set, using in hyper variable tuning and permutation importance
                X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3) # 70% training and 30% tes

                # Define the Random Forest Classifier
                rfclassifier = RandomForestClassifier(random_state = 42)

                """
                shuffle split then calculate the cross_validation TSS score
                """
                cv = ShuffleSplit(n_splits=5, test_size=0.3, random_state=0) # 70% training set

                best_cross_score = np.mean(cross_val_score(rfclassifier, X, y, cv=cv,scoring=tss_scorer))
                count_average+=best_cross_score


                """
                Train the RF model
                """
                n_repeats=100

                print("length_ytest: "+str(len(y_test)))
                
                rfclassifier.fit(X_train,y_train)
                


                """
                mean-decrease in gini impurity importance
                """ 

                gini_importances = rfclassifier.feature_importances_
                gini_std = np.std([tree.feature_importances_ for tree in rfclassifier.estimators_], axis=0)

                ginithis_importance_score = pd.Series(gini_importances, index=x_labels)
                giniarray_list = np.append(giniarray_list, np.array(ginithis_importance_score))
                ginithis_importance_variance = np.array(gini_std)
                ginivariance_list = np.append(ginivariance_list,ginithis_importance_variance)


                """
                Boruta
                """
                blockPrint()
                # define Boruta feature selection method
                this_random_forest = deepcopy(rfclassifier)
                feat_selector = BorutaPy(this_random_forest, n_estimators='auto', verbose=2, random_state=42)

                X_train_np = np.array(X_train)
                y_train_np = np.array(y_train)


                # find all relevant features
                feat_selector.fit(X_train_np, y_train_np)

                # call transform() on X to filter it down to selected features
                X_filtered = feat_selector.transform(X_train_np)

                feature_ranks = np.array(list(zip(x_labels,feat_selector.ranking_, feat_selector.support_)))

                # iterate through and print out the results

                for feat in feature_ranks:
                    print('Feature: {:<25} Rank: {},  Keep: {}'.format(feat[0], feat[1], feat[2]))


                boruta_rank_array = feature_ranks[:,1].astype('float')
                boruta_rank = np.append(boruta_rank,boruta_rank_array)


                enablePrint()

                """
                Shapley value
                """
                shap_random_forest = deepcopy(rfclassifier)
                shap_random_forest1 = deepcopy(rfclassifier)
                shap_values = shap.TreeExplainer(shap_random_forest).shap_values(X_train)

                shape_values_list0 = np.append(shape_values_list0, shap_values[0])
                shape_values_list1 = np.append(shape_values_list1, shap_values[1])
                
                x_train_length = X_train.shape[0]
                x_train_list = np.append(x_train_list, np.array(X_train))


                """
                Record the tree
                """
                tree_filename = save_tree_dir+'/'+name_for_saving+'_'+str(count_num)+'iters.sav'
                joblib.dump(rfclassifier, tree_filename)

                """
                Test on the whole North America
                """
                final_testing_thisround = final_testing_group.copy()
                final_prediction=rfclassifier.predict(final_testing_thisround)
                predict_list = np.append(predict_list, final_prediction)

                """
                Compute ROC and AUC
                """
                # Compute ROC curve and ROC area for each class
                y_score = rfclassifier.predict_proba(X_test)[:,1]

                fpr, tpr, _ = roc_curve(y_test, y_score)

                roc_auc = auc(fpr, tpr)


                roc_auc_count = roc_auc_count+ roc_auc

                n = 1000
                x_interp = np.linspace(0,1,n)
                fpr_list = np.append(fpr_list, x_interp)
                y_interp = np.interp(x_interp,fpr, tpr)
                tpr_list = np.append(tpr_list, y_interp)


                count_num+=1








            #save everything in a dictionary

            #cross validation
            cross_val_score_ave = count_average/num_runs

            #gini importance

            giniarray_list1 = giniarray_list.reshape(num_runs, variable_length)
            giniarray_list1 = np.mean(giniarray_list1, axis=0)

            ginivariance_list0 = ginivariance_list.reshape(num_runs, variable_length)
            ginivariance_list0 = np.mean(ginivariance_list0,axis=0)

            #ROC
            #fpr
            fpr_list1 = fpr_list.reshape(num_runs, 1000)
            #tpr
            tpr_list1 = tpr_list.reshape(num_runs, 1000)
            
            #auc
            roc_auc_count_ave = roc_auc_count/num_runs


            #boruta
            boruta_rank1 = boruta_rank.reshape(num_runs, variable_length)
            boruta_rank1 = np.mean(boruta_rank1,axis=0)

            #shapely

            shape_values_list00 = np.mean(shape_values_list0.reshape(x_train_length*num_runs,variable_length), axis=0)
            shape_values_list11 = np.mean(shape_values_list1.reshape(x_train_length*num_runs,variable_length), axis=0)



            x_train_list001 = x_train_list.reshape(x_train_length*num_runs,variable_length)
            x_train_list1 = pd.DataFrame(x_train_list001, columns = x_labels)

            dict_save = {'cross_validation_score' : cross_val_score_ave, 
                    'x_label' : x_labels, 
                    'X_list':X_list,
                    'y_list':y_list,
                    'number_of_presence':number_of_presence,
                    'gini_importance_score' : giniarray_list1,
                    'gini_importance_variance' : ginivariance_list0,
                    'boruta_rank':boruta_rank1,
                    'auc':roc_auc_count_ave,
                    'fpr':fpr_list1,
                    'tpr':tpr_list1,
                    'x_train_list1':x_train_list1,
                    'shape_values_list0':shape_values_list00,
                    'shape_values_list1':shape_values_list11}
            
            dictionary_path = save_tree_dir + '/'+name_for_saving+'_dictionary_'+str(num_runs)+'iters.pickle'
            with open(dictionary_path, 'wb') as handle:
                pickle.dump(dict_save, handle, protocol=pickle.HIGHEST_PROTOCOL)



            with open(dictionary_path, 'rb') as handle:
                dict_read = pickle.load(handle)


            """
            ROC
            """

            fpr_plot = dict_read['fpr']
            tpr_plot = dict_read['tpr']
            #print(tpr_plot)
            plt.figure(figsize=(10,10))
            lw = 2

            for i in range(len(tpr_plot)):
                plt.plot(fpr_plot[0],tpr_plot[i])

            plt.xlim([-0.05, 1.0])
            plt.ylim([0.0, 1.05])
            plt.xlabel("False Positive Rate",fontsize = 15)
            plt.ylabel("True Positive Rate",fontsize = 15)
            plt.tick_params(axis='x', labelsize= 15 )
            plt.tick_params(axis='y', labelsize= 15 )
            plt.title('ROC '+ name_for_saving+' ('+str(num_runs)+' iters)',fontsize = 20)

            #create a directory for storing the ROC
            ROC_dir = '/Users/huangliting/Desktop/kissing_bugs_stats/ROC/'+name_for_saving

            #print(ROC_dir)

            if not os.path.exists(ROC_dir):
                os.makedirs(ROC_dir)


            store_graph_name = ROC_dir+'/'+name_for_saving+str(num_runs)+'iters.png'
            #print(store_graph_name)
            plt.savefig(store_graph_name)


            print("cross_validation: "+str(cross_val_score_ave))
            print("auc: "+str(roc_auc_count_ave))
            plt.clf()

            """
            gini
            """

            giniforest_importance1 = pd.Series(giniarray_list1, index=x_labels)

            #print(forest_importance1)

            fig, ax = plt.subplots(figsize=(20,20))

            giniforest_importance1.plot.bar(yerr=ginivariance_list0, ax=ax, fontsize=30)
            ax.set_title("mean decrease gini impurity importances " + name_for_saving+' ('+str(num_runs)+' iters)',fontsize=30)
            ax.set_ylabel("Mean decrease in impurity",fontsize=30)
            plt.grid()
            fig.tight_layout()

            #create a directory for storing the trees
            gini_dir = '/Users/huangliting/Desktop/kissing_bugs_stats/gini_importance/'+name_for_saving

            #print(gini_dir)

            if not os.path.exists(gini_dir):
                os.makedirs(gini_dir)


            store_graph_name = gini_dir+'/'+name_for_saving+'_'+str(num_runs)+'iters.png'
            #print(store_graph_name)
            plt.savefig(store_graph_name)
            plt.clf()




            """
            boruta graph
            """
            boruta_importance1 = pd.Series(boruta_rank1, index=x_labels)
            fig, ax = plt.subplots(figsize=(20,20))

            boruta_importance1.plot.bar(ax=ax, fontsize=30)
            ax.set_title("boruta importances " + name_for_saving+' ('+str(num_runs)+' iters)',fontsize=30)
            ax.set_ylabel("Mean rank",fontsize=30)
            plt.grid()
            fig.tight_layout()


            #create a directory for storing the trees
            boruta_dir = '/Users/huangliting/Desktop/kissing_bugs_stats/boruta_importance/'+name_for_saving


            if not os.path.exists(boruta_dir):
                os.makedirs(boruta_dir)


            store_graph_name = boruta_dir+'/'+name_for_saving+'_'+str(num_runs)+'iters.png'
            #print(store_graph_name)
            plt.savefig(boruta_dir+'/'+name_for_saving+'_'+str(num_runs)+'iters.png')
            plt.clf()





            """
            shapely graph
            """
            fig, ax = plt.subplots(figsize=(60,20))
            x_shaply_value = np.arange(len(x_labels))  # the label locations
            width = 0.35  # the width of the bars


            rects1 = ax.bar(x_shaply_value - width/2, np.abs(shape_values_list00), width, label='absence')
            rects2 = ax.bar(x_shaply_value + width/2, np.abs(shape_values_list11), width, label='presence')

            # Add some text for labels, title and custom x-axis tick labels, etc.
            ax.set_ylabel('Scores',fontsize=40,)
            ax.set_title('Scores by group and presence-absence')
            ax.set_xticks(x_shaply_value, x_labels,fontsize=40,rotation='vertical')
            ax.legend(fontsize=40)

            # ax.bar_label(rects1, padding=3)
            # ax.bar_label(rects2, padding=3)
            # ax.tick_params(axis='x', labelsize= 40 )
            ax.tick_params(axis='y', labelsize= 40 )
            #shap.summary_plot(shape_values_list, x_train_list1, plot_type="bar",max_display=variable_length)
            ax.set_title("absolute shapely values " + name_for_saving+' ('+str(num_runs)+' iters)',fontsize=40)
            plt.grid()
            fig.tight_layout()


            shap_dir = '/Users/huangliting/Desktop/kissing_bugs_stats/shap_importance/'+name_for_saving


            if not os.path.exists(shap_dir):
                os.makedirs(shap_dir)


            store_graph_name = shap_dir+'/'+name_for_saving+'_'+str(num_runs)+'iters.png'
            print(store_graph_name)
            plt.savefig(shap_dir+'/'+name_for_saving+'_'+str(num_runs)+'iters.png')
            plt.clf()

            """
            save prediction
            """

            #obtain average prediction list
            testing_group_size_pre = len(final_testing_group['MSP'])
            prediction_final_list = np.mean(predict_list.reshape(num_runs, testing_group_size_pre), axis=0)

            # initialize data of lists.
            final_list_data_f = {'id': id_list,
                    'prediction': prediction_final_list}

            # Create DataFrame
            dataframe_fial_prediction = pd.DataFrame(final_list_data_f)
            """change_name"""
            #create a directory for storing the trees
            pre_dir = '/Users/huangliting/Desktop/kissing_bugs_stats/predictions/'+name_for_saving

            #print(pre_dir)

            if not os.path.exists(pre_dir):
                os.makedirs(pre_dir)


            dataframe_fial_prediction.to_csv( pre_dir+'/'+name_for_saving+'_'+str(num_runs)+'iters.csv')
            #print(pre_dir+'/'+name_for_saving+'_'+str(num_runs)+'iters.csv')
            """
            write txt
            """

            with open('/Users/huangliting/Desktop/kissing_bugs_stats/txt_results/'+name_for_saving+'_'+str(num_runs)+'iters.txt', 'a') as f:
                f.write(name_for_saving+'\n')
                f.write('number of presence points: '+str(number_of_presence)+'\n')
                f.write('cross validation score: '+str(cross_val_score_ave)+'\n')
                f.write('auc: '+str(roc_auc_count_ave)+'\n')